In [1]:
!pip install dvc

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 458.5/458.5 kB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.3/77.3 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.8/438.8 kB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.0/210.0 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.6/118.6 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.1/73.1 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.9/374.9 kB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8

In [2]:
!git init
!dvc init -f

hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
Initialized empty Git repository in /content/.git/
Initialized DVC repository.

You can now commit the changes to git.

+---------------------------------------------------------------------+
|                                                                     |
|        DVC has enabled anonymous aggregate usage analytics.         |
|     Read the analytics documentation (and how to opt-out) here:     |
|             <https://dvc.org/doc/user-guide/analytics>              |
|                                  

In [3]:
!mkdir data models
!mkdir -p scripts

In [4]:
%%writefile scripts/download_data.py
import pandas as pd
from sklearn.datasets import load_iris

df = pd.DataFrame(load_iris(as_frame=True).frame)
df.to_csv("data/iris.csv", index=False)

Writing scripts/download_data.py


In [5]:
%%writefile scripts/train_model.py
import pandas as pd
import pickle
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import yaml

with open("params.yaml") as f:
    params = yaml.safe_load(f)

df = pd.read_csv("data/iris.csv")
X = df.drop("target", axis=1)
y = df["target"]
X_train, _, y_train, _ = train_test_split(X, y, test_size=params["train"]["test_size"])

model = LogisticRegression(max_iter=params["train"]["max_iter"])
model.fit(X_train, y_train)

with open("models/model.pkl", "wb") as f:
    pickle.dump(model, f)

Writing scripts/train_model.py


In [6]:
%%writefile scripts/evaluate_model.py
import pandas as pd
import pickle
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

df = pd.read_csv("data/iris.csv")
X = df.drop("target", axis=1)
y = df["target"]
_, X_test, _, y_test = train_test_split(X, y, test_size=0.2)

with open("models/model.pkl", "rb") as f:
    model = pickle.load(f)

y_pred = model.predict(X_test)
acc = accuracy_score(y_test, y_pred)

with open("metrics.txt", "w") as f:
    f.write(f"Accuracy: {acc:.4f}\n")

Writing scripts/evaluate_model.py


In [7]:
%%writefile params.yaml
train:
  test_size: 0.2
  max_iter: 200

Writing params.yaml


In [8]:
!dvc stage add -n download_data \
  -d scripts/download_data.py \
  -o data/iris.csv \
  python scripts/download_data.py

Added stage 'download_data' in 'dvc.yaml'

To track the changes with git, run:

	git add dvc.yaml data/.gitignore

To enable auto staging, run:

	dvc config core.autostage true


In [9]:
!dvc stage add -n train_model \
  -d scripts/train_model.py -d data/iris.csv -d params.yaml \
  -o models/model.pkl \
  python scripts/train_model.py

Added stage 'train_model' in 'dvc.yaml'

To track the changes with git, run:

	git add models/.gitignore dvc.yaml

To enable auto staging, run:

	dvc config core.autostage true


In [10]:
!dvc stage add -n evaluate_model \
  -d scripts/evaluate_model.py -d models/model.pkl -d data/iris.csv \
  -o metrics.txt \
  python scripts/evaluate_model.py

Added stage 'evaluate_model' in 'dvc.yaml'

To track the changes with git, run:

	git add .gitignore dvc.yaml

To enable auto staging, run:

	dvc config core.autostage true


In [11]:
!dvc params diff

In [12]:
!dvc repro

Running stage 'download_data':
> python scripts/download_data.py
Generating lock file 'dvc.lock'
Updating lock file 'dvc.lock'

Running stage 'train_model':
> python scripts/train_model.py
Updating lock file 'dvc.lock'

Running stage 'evaluate_model':
> python scripts/evaluate_model.py
Updating lock file 'dvc.lock'

To track the changes with git, run:

	git add dvc.lock

To enable auto staging, run:

	dvc config core.autostage true
Use `dvc push` to send your updates to remote storage.


In [13]:
%%writefile params.yaml
train:
  test_size: 0.25
  max_iter: 300

Overwriting params.yaml


In [14]:
!dvc repro

Stage 'download_data' didn't change, skipping
Running stage 'train_model':
> python scripts/train_model.py
Updating lock file 'dvc.lock'

Running stage 'evaluate_model':
> python scripts/evaluate_model.py
Updating lock file 'dvc.lock'

To track the changes with git, run:

	git add dvc.lock

To enable auto staging, run:

	dvc config core.autostage true
Use `dvc push` to send your updates to remote storage.


In [15]:
!mkdir lab3
!mv scripts lab3/
!mv data lab3/
!mv models lab3/
!mv params.yaml lab3/
!mv dvc.yaml lab3/
!mv dvc.lock lab3/
!mv metrics.txt lab3/
